In [1]:
import numpy as np
import pandas as pd

In [2]:
books = pd.read_csv('Books.csv')
books.head(5)

C:\Users\gyash\AppData\Local\Temp\ipykernel_7468\3147312277.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv('Books.csv')


,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [3]:
books.drop(['Image-URL-S','Image-URL-M'],axis=1,inplace=True)
books.rename(columns={'Image-URL-L':'Cover Page','ISBN':'Book ID'}, inplace=True)
books.head(5)

,Book ID,Book-Title,Book-Author,Year-Of-Publication,Publisher,Cover Page
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...


In [4]:
books.dtypes

Book ID                object
Book-Title             object
Book-Author            object
Year-Of-Publication    object
Publisher              object
Cover Page             object
dtype: object

In [5]:
valid_years = books['Year-Of-Publication'].astype(str).str.isnumeric()
books = books[valid_years]
books['Year-Of-Publication'] = books['Year-Of-Publication'].astype(int)

In [6]:
books['Mod-Title']=books['Book-Title'].str.replace("[^a-zA-Z0-9 ]", "", regex=True)

In [7]:
books['Mod-Title']=books['Mod-Title'].str.lower()

In [8]:
books['Mod-Title']=books['Mod-Title'].str.replace("\s+", " ", regex=True)

In [9]:
books=books[books['Mod-Title'].str.len()>0]

In [10]:
books.head(5)

,Book ID,Book-Title,Book-Author,Year-Of-Publication,Publisher,Cover Page,Mod-Title
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,classical mythology
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,clara callan
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,decision in normandy
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,flu the story of the great influenza pandemic ...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,the mummies of urumchi


In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()

tfidf = vectorizer.fit_transform(books["Mod-Title"])

In [12]:
from sklearn.metrics.pairwise import cosine_similarity

In [13]:
def show_image(val):
    return '<a href="{}"><img src="{}" width=50></img></a>'.format(val, val)

In [14]:
import re

In [15]:
def search(title,vectorizer):
    mod_title=re.sub("[^a-zA-Z0-9 ]", "", title.lower())
    title_vec = vectorizer.transform([mod_title])
    similarity = cosine_similarity(title_vec,tfidf).flatten()

    indices = np.argpartition(similarity,-10)[-10:]

    suggestion = books.iloc[indices]
    suggestion = suggestion.sort_values("Year-Of-Publication", ascending=False)
    suggestion.drop(['Mod-Title'],axis=1,inplace=True)
    return suggestion.style.format({'Cover Page':show_image})

In [16]:
search('POtter',vectorizer)

,Book ID,Book-Title,Book-Author,Year-Of-Publication,Publisher,Cover Page
35086,043965548X,Harry Potter and the Prisoner of Azkaban (Harry Potter),J.K. Rowling,2004,Scholastic Paperbacks,
140443,1594130027,Harry Potter and the Prisoner of Azkaban (Harry Potter (Paperback)),J. K. Rowling,2003,Large Print Press,
257263,043955490X,Harry Potter and the Goblet of Fire (Harry Potter),J. K. Rowling,2003,Arthur A. Levine Books,
72186,0439554896,Harry Potter and the Chamber of Secrets (Harry Potter),J. K. Rowling,2003,Arthur A. Levine Books,
95841,0374214948,Mr. Potter: A Novel,Jamaica Kincaid,2002,Farrar Straus Giroux,
209283,0613206339,Harry Potter and the Sorcerer's Stone (Harry Potter (Hardcover)),J. K. Rowling,2001,Sagebrush Bound,
52580,031226481X,We Love Harry Potter!,Sharon Moore,1999,St. Martin's Press,
77384,059035342x,Harry Potter and the Sorcerer's Stone (Harry Potter (Paperback)),J. K. Rowling,1999,Arthur A. Levine Books,
2143,059035342X,Harry Potter and the Sorcerer's Stone (Harry Potter (Paperback)),J. K. Rowling,1999,Arthur A. Levine Books,
164470,1550500376,The Potter,Jacolyn Caton,1992,Coteau Books,


In [17]:
search('Alchemist',vectorizer)

,Book ID,Book-Title,Book-Author,Year-Of-Publication,Publisher,Cover Page
258305,034546236X,The Alchemist,Donna Boyd,2003,Ballantine Books,
129271,0345441141,The Alchemist,DONNA BOYD,2002,Ballantine Books,
201515,8485982665,The Alchemist,Jorge A. Livraga Rizzi,2001,New Acropolis Cultural Assn,
153724,0694524476,"Alchemist, The",Paulo Coelho,2001,HarperAudio,
51665,0752817299,Alchemist,Peter James,1999,Orion Publishing Group,
2472,0553476580,Alchemist,Paulo Coelho,1995,Random House Audio,
197978,0393900568,The Alchemist,Ben Jonson,1991,W. W. Norton & Company,
57670,0747504687,The Alchemist,Mark Illis,1990,Bloomsbury Publishing Plc,
40015,0553255983,The Alchemist,Ken Goddard,1986,Bantam Books,
217188,0380000784,The Alchemist,Leslie H. Whitten,1976,Avon Books,


In [19]:
search('rich dad',vectorizer)

,Book ID,Book-Title,Book-Author,Year-Of-Publication,Publisher,Cover Page
101,0446677450,"Rich Dad, Poor Dad: What the Rich Teach Their Kids About Money--That the Poor and Middle Class Do Not!",Robert T. Kiyosaki,2000,Warner Books,
253502,155704256X,Dad: A Novel,William Wharton,1996,Newmarket Press,
105702,0440503027,How to Dad,John Boswell,1990,Dell Publishing Company,
201328,0380585944,Dad,William Wharton,1989,Avon Books,
103482,0517565730,American Dad,Tama Janowitz,1987,Random House Inc,
255777,0836279271,"Love, Dad",Patrick Connolly,1985,Andrews McMeel Pub,
146416,0440149983,"Love, Dad",Evan Hunter,1982,Dell Publishing Company,
267933,0394510976,Dad: A Novel,William Wharton,1981,Random House Inc,
103043,039912585X,American dad,Tama Janowitz,1981,Putnam,
170380,0771091249,Rich Man,Kreisel,1961,McClelland & Stewart,
